In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.options import Options
import csv
import pandas as pd
import urllib
import ssl
import random


search = input('검색어 : ')
search1 = urllib.parse.quote(search)
url = 'https://www.instagram.com/explore/tags/'+str(search1)+'/'

driver = webdriver.Chrome('C:/3pm/pythonwork/analysis/chromedriver/chromedriver') # 크롬드라이버 위치 수정
driver.implicitly_wait(1)
driver.get(url)
time.sleep(20)

driver.get(url)
time.sleep(5)
driver.find_element_by_class_name('eLAPa').click()

time.sleep(2)

with open(search+'인스타.csv', 'a', newline='', encoding='utf-8') as out:
    writer = csv.writer(out)
    replylist = []
    for temp in range(0,29):
        html = driver.page_source
        soup = BeautifulSoup(html, 'lxml')
        #result = soup.find_all('a', {'class': 'xil3i'})  #해시태그가 있는 클래스
        rdata = soup.find_all('div', {'class': 'C4VMK'})  #댓글이 있는 클래스

        if len(rdata) > 0:  #댓글이 있으면
            hstr = ''   #게시글 하나의 댓글을 하나의 문장으로 저장한다.
            rresult = []
            for temp1 in rdata:
                temp1 = temp1.find_all('span')[1]
                print(temp1)
                hstr = hstr + ' ' + temp1.text
            replylist.append(hstr)
            rresult.append(hstr)

            datelist = list()
            time_raw = driver.find_element_by_css_selector('body > div._2dDPU.CkGkG > div.zZYga > div > article > div.eo2As > div.k_Q0X.I0_K8.NnvRN > a > time')
            time_info = pd.to_datetime(time_raw.get_attribute('datetime')).normalize()
            datelist.append(time_info)
            print(datelist)

            writer.writerow(datelist + rresult)
        driver.find_element_by_link_text('다음').click()  #다음 클릭
        time.sleep(2) #로딩이 느리면 시간을 좀 늘려주기 #봇으로 인식하는걸 막기 위해 랜덤으로 넣어줌
            #random.randint(3,10)

print(search1)
print(replylist)
print(datelist)

In [ ]:
data = pd.read_csv("갤럭시인스타댓글.csv", header = None)

In [ ]:
def get_tags(text, ntags=200):
    spliter = Okt()
    nouns = spliter.nouns(text)
    count = Counter(nouns)
    return_list = []
    for n, c in count.most_common(ntags):  #count로 단어의 갯수를 파악한걸 정렬해서 ntags갯수만큼 리스트로 가져온다.
        temp = {'tag': n, 'count': c}
        return_list.append(temp)
    # return return_list
    return dict(count.most_common(ntags))       #[(커피, 1200),(카페, 800)] -> {커피:1200, 카페:800}

def clean_str(s):
    hangul = re.compile('[^ ㄱ-ㅣ가-힣]+')    #한글만 남겨놓고 나머지 삭제
    s = hangul.sub(r'',str(s))

    cp = re.compile("["u"\U00010000-\U0010FFFF""]+", flags=re.UNICODE)  #BMP characters 이외 이모티콘삭제
    s=cp.sub(r'',s)

    s = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]', '', s) #특수기호 삭제
 
    return s.strip()

In [ ]:
clean_data = clean_str(data[1])